In [ ]:
import requests
import json

# Çekilecek API adresi (örnek: sahte kullanıcı verisi)
api_url = "https://jsonplaceholder.typicode.com/users"

try:
    # API'ye GET isteği gönderme
    response = requests.get(api_url)

    # İstek başarılı olduysa (HTTP 200)
    if response.status_code == 200:
        # JSON verisini Python listesine/sözlüğüne dönüştürme
        users_data = response.json()

        print(f"✅ Başarılı! {len(users_data)} kullanıcı verisi çekildi.")
        print("-" * 30)
        # İlk kullanıcının ismini ve e-postasını yazdır
        print(f"İlk Kullanıcı Adı: {users_data[0]['name']}")
        print(f"İlk Kullanıcı E-posta: {users_data[0]['email']}")

    else:
        print(f"❌ API isteği başarısız oldu. Durum kodu: {response.status_code}")

except requests.exceptions.RequestException as e:
    print(f"❌ Bir hata oluştu: {e}")

✅ Başarılı! 10 kullanıcı verisi çekildi.
------------------------------
İlk Kullanıcı Adı: Leanne Graham
İlk Kullanıcı E-posta: Sincere@april.biz


In [ ]:
import pandas as pd
import numpy as np

# Örnek bir DataFrame oluşturma (normalde pd.read_csv() ile yüklenir)
data = {
    'UrunID': [101, 102, 103, 104, 105],
    'Fiyat': [15.50, 22.00, np.nan, 30.75, 12.00],
    'Kategori': ['A', 'B', 'A', 'C', 'B'],
    'Stok': [100, 50, 120, np.nan, 80],
    'Satildi': [True, False, True, True, False]
}
df = pd.DataFrame(data)

print("### 1. Başlangıç Veri Çerçevesi ###")
print(df)
print("-" * 35)

# 1. Veri Keşfi
print("### 2. Eksik Veri Kontrolü ###")
print(df.isna().sum())
print("-" * 35)

# 2. Eksik Veri İşleme
# 'Fiyat' sütunundaki eksik değeri ortalama ile doldurma
ortalama_fiyat = df['Fiyat'].mean()
df['Fiyat'].fillna(ortalama_fiyat, inplace=True)

# 'Stok' sütunundaki eksik değeri 0 ile doldurma
df['Stok'].fillna(0, inplace=True)

print("### 3. Eksik Veri Sonrası Durum (Dolduruldu) ###")
print(df)
print("-" * 35)

# 3. Gruplama ve Agregasyon
# Kategoriye göre ortalama fiyatı hesaplama
ortalama_fiyat_kategori = df.groupby('Kategori')['Fiyat'].mean().reset_index()

print("### 4. Kategoriye Göre Ortalama Fiyat ###")
print(ortalama_fiyat_kategori)

### 1. Başlangıç Veri Çerçevesi ###
   UrunID  Fiyat Kategori   Stok  Satildi
0     101  15.50        A  100.0     True
1     102  22.00        B   50.0    False
2     103    NaN        A  120.0     True
3     104  30.75        C    NaN     True
4     105  12.00        B   80.0    False
-----------------------------------
### 2. Eksik Veri Kontrolü ###
UrunID      0
Fiyat       1
Kategori    0
Stok        1
Satildi     0
dtype: int64
-----------------------------------
### 3. Eksik Veri Sonrası Durum (Dolduruldu) ###
   UrunID    Fiyat Kategori   Stok  Satildi
0     101  15.5000        A  100.0     True
1     102  22.0000        B   50.0    False
2     103  20.0625        A  120.0     True
3     104  30.7500        C    0.0     True
4     105  12.0000        B   80.0    False
-----------------------------------
### 4. Kategoriye Göre Ortalama Fiyat ###
  Kategori     Fiyat
0        A  17.78125
1        B  17.00000
2        C  30.75000


/tmp/ipython-input-3981690187.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Fiyat'].fillna(ortalama_fiyat, inplace=True)
/tmp/ipython-input-3981690187.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try 

In [ ]:
# Kütüphaneyi yükle
from datasets import load_dataset, Dataset

# Hugging Face Hub'dan 'imdb' (duygu analizi) veri setini yükle
# 'train' ve 'test' bölümlerini içerir
raw_datasets = load_dataset("imdb")

print("### 1. Yüklenen Veri Seti Yapısı ###")
print(raw_datasets)
print("-" * 30)

# Veri setindeki bir örneği inceleyelim
print("### 2. İlk Eğitim Örneği (Raw) ###")
print(raw_datasets['train'][0])
print("-" * 30)

# Veri işleme (Haritalama/Mapping) Fonksiyonu
# Bu basit örnekte, metnin uzunluğunu hesaplayacağız.
# Gerçek NLP'de burada 'tokenization' (kelimelere/alt kelimelere ayırma) yapılır.
def calculate_length(example):
    """Metin uzunluğunu hesaplar ve yeni bir sütun olarak ekler."""
    example["text_length"] = len(example["text"].split()) # Kelime sayısını al
    return example

# map() fonksiyonunu kullanarak fonksiyonu tüm veri setine uygulama
# batched=True ve num_proc > 1 kullanmak büyük veri setlerinde performansı artırır.
processed_datasets = raw_datasets.map(
    calculate_length,
    num_proc=1, # Basitlik için 1, gerçek projede CPU çekirdek sayısı kadar kullanılabilir
    batched=False # Tekil örneklere uygulama (kelime sayımı)
)

print("### 3. İşlenmiş İlk Eğitim Örneği (Mapped) ###")
print(processed_datasets['train'][0])
print("-" * 30)

# Oluşan yeni sütunu kontrol etme
print("Yeni Oluşan 'text_length' Sütunu Ortalaması:")
# Bir HF Dataset nesnesindeki bir sütunun değerlerini almak için
# processed_datasets['train']['text_length'] kullanılır.
avg_length = sum(processed_datasets['train']['text_length']) / len(processed_datasets['train'])
print(f"Ortalama Kelime Sayısı: {avg_length:.2f}")

### 1. Yüklenen Veri Seti Yapısı ###
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})
------------------------------
### 2. İlk Eğitim Örneği (Raw) ###
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political